In [2]:
!python3 --version

Python 3.11.3


In [4]:
!pip3 install numpy matplotlib


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 35.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 69.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.6/254.6 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 71.2 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 71.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.1/111.1 kB 12.8 MB/s eta 0:00:00

[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [9]:
import numpy as np
import matplotlib.pyplot as plt
from itertools import permutations
import time

# Generate random city coordinates
# n_cities defaults to 6
def generate_cities(n_cities=6, seed=42):
    np.random.seed(seed)
    return np.random.rand(n_cities, 2)

# Distance matrix
def compute_distance_matrix(cities):
    n = len(cities)
    dist = np.zeros((n, n))
    for i in range(n):
        for j in range(i + 1, n):
            d = np.linalg.norm(cities[i] - cities[j])
            dist[i, j] = dist[j, i] = d
    return dist

# Total route distance
def route_distance(route, dist_matrix):
    return sum(dist_matrix[route[i], route[(i+1)%len(route)]] for i in range(len(route)))

# 1. Brute-force TSP
# This algorithm finds the shortest possible route by trying every single possible permutation of cities.
# It's guaranteed to find the best route, but it's very slow for a large number of cities.
def tsp_brute_force(dist_matrix):
    n = len(dist_matrix)
    shortest_route = None
    min_distance = float('inf')
    for perm in permutations(range(1, n)):
        route = [0] + list(perm)
        d = route_distance(route, dist_matrix)
        if d < min_distance:
            min_distance = d
            shortest_route = route
    return shortest_route, min_distance

# 2. Nearest Neighbor
# This is a "greedy" algorithm. It starts at a random city and repeatedly visits the nearest unvisited city.
# It's much faster than brute-force, but it often doesn't find the best route.
def tsp_nearest_neighbor(dist_matrix):
    n = len(dist_matrix)
    unvisited = set(range(n))
    route = [0]
    unvisited.remove(0)
    while unvisited:
        last = route[-1]
        next_city = min(unvisited, key=lambda city: dist_matrix[last][city])
        route.append(next_city)
        unvisited.remove(next_city)
    return route, route_distance(route, dist_matrix)

# 3. 2-opt Swap
# This algorithm starts with a route (e.g., from Nearest Neighbor) and tries to improve it.
# It repeatedly swaps pairs of edges to see if it can find a shorter route.
def two_opt(route, dist_matrix):
    improved = True
    best = route[:]
    best_distance = route_distance(best, dist_matrix)
    while improved:
        improved = False
        for i in range(1, len(route) - 2):
            for j in range(i + 1, len(route)):
                if j - i == 1: continue
                new_route = best[:]
                new_route[i:j] = best[j-1:i-1:-1]
                new_dist = route_distance(new_route, dist_matrix)
                if new_dist < best_distance:
                    best = new_route
                    best_distance = new_dist
                    improved = True
        route = best[:]
    return best, best_distance

# Run the comparison
def compare_algorithms(n_cities=6):
    cities = generate_cities(n_cities)
    dist_matrix = compute_distance_matrix(cities)

    results = {}
    routes = {}

    if n_cities > 9:
        print("Warning: Brute-force algorithm may be very slow for more than 9 cities.")

    t0 = time.time()
    brute_route, brute_dist = tsp_brute_force(dist_matrix)
    t1 = time.time()
    results["Brute Force"] = (brute_dist, t1 - t0)
    routes["Brute Force"] = brute_route

    t0 = time.time()
    nn_route, nn_dist = tsp_nearest_neighbor(dist_matrix)
    t1 = time.time()
    results["Nearest Neighbor"] = (nn_dist, t1 - t0)
    routes["Nearest Neighbor"] = nn_route

    t0 = time.time()
    opt_route, opt_dist = two_opt(nn_route, dist_matrix)
    t1 = time.time()
    results["2-opt Swap"] = (opt_dist, t1 - t0)
    routes["2-opt Swap"] = opt_route

    return cities, results, routes

# Example usage:
# You can now easily change the number of cities by modifying the 'n_cities' variable.
n_cities = 10
cities, results, routes = compare_algorithms(n_cities)
print(f"Results for {n_cities} cities:")
for name, (dist, duration) in results.items():
    print(f"{name}: distance = {dist:.4f}, time = {duration:.4f} sec")

#Results for 6 cities:
#Brute Force: distance = 2.4107, time = 0.0004 sec
#Nearest Neighbor: distance = 2.7959, time = 0.0000 sec
#2-opt Swap: distance = 2.7516, time = 0.0001 sec

#Results for 9 cities:
#Brute Force: distance = 2.8665, time = 0.3480 sec
#Nearest Neighbor: distance = 3.3975, time = 0.0001 sec
#2-opt Swap: distance = 3.3175, time = 0.0004 sec

#Warning: Brute-force algorithm may be very slow for more than 9 cities.
#Results for 10 cities:
#Brute Force: distance = 2.9031, time = 1.5826 sec
#Nearest Neighbor: distance = 3.1224, time = 0.0001 sec
#2-opt Swap: distance = 2.9031, time = 0.0003 sec

#Warning: Brute-force algorithm may be very slow for more than 9 cities.
#Results for 11 cities:
#Brute Force: distance = 2.9625, time = 18.7909 sec
#Nearest Neighbor: distance = 3.8028, time = 0.0001 sec
#2-opt Swap: distance = 3.5584, time = 0.0004 sec

#Warning: Brute-force algorithm may be very slow for more than 9 cities.
#Results for 12 cities:
#Brute Force: distance = 2.9725, time = 233.4507 sec
#Nearest Neighbor: distance = 3.8542, time = 0.0001 sec
#2-opt Swap: distance = 3.5683, time = 0.0005 sec

Results for 10 cities:
Brute Force: distance = 2.9031, time = 0.7675 sec
Nearest Neighbor: distance = 3.1224, time = 0.0000 sec
2-opt Swap: distance = 2.9031, time = 0.0001 sec
